/*************************************************************************************************************************
                                                     ##Disclaimer##
    
-> We are providing this code block strictly for learning and researching, this is not a production ready code. 
-> We have no liability on this particular code under any circumstances; users should use this code on their own risk. 

-> All software, hardware and othr products that are referenced in these materials belong to the respective vendor who developed or who owns this product.

/*************************************************************************************************************************
                                                  @DeepSphere.AI ,Inc

# Step 0: downloading data from storage bucket

In [ ]:
from google.cloud import storage

# Initialise a client
storage_client = storage.Client("medicaltranscript ")
# Create a bucket object for our bucket
bucket = storage_client.get_bucket('medicaltranscript')
# Create a blob object from the filepath
blob = bucket.blob("deepspeech_file.zip")
# Download the file to a destination
blob.download_to_filename('deepspeech_file.zip')

!unzip deepspeech_file

# Step 1: .YAML File Configuration

In [ ]:
import configparser
# Importing the yaml library

import yaml
# Importing the OS library

import os

vAR_yaml_file_path = '/home/jupyter/MT/Healthcare/MedicalTranscription/4_ModelImplementation/config.yaml'
    
with open (vAR_yaml_file_path) as vAR_file:
    
           config = yaml.safe_load(vAR_file)

In [ ]:
vAR_Text_Path=config.get("text_file")
vAR_Model_Path=config.get("model_file_path")
vAR_LM_Path=config.get("lm_file_path")
vAR_Symptom_Path=config.get("symptoms_path")

# Step 2: Importing the Required Libraries

In [ ]:
import nltk
import configparser
import matplotlib.pyplot as plt
from pandas import DataFrame
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import cufflinks as cf
import plotly.offline
import pandas as pd
import numpy as np
from deepspeech import Model 
import numpy as np
import wave
import git

# Step 3: Importing DeepSpeech model for convert audio to text

In [ ]:
def main():  
    Voice_to_text()
    Clone_CliNER()
    Predict_symptoms()
    Text_Classification_and_Visualization()
    
def Voice_to_text():
    vAR_beam_width=1000
    vAR_lm_alpha=0.93 
    vAR_lm_beta=1.18

    model=Model(vAR_Model_Path)
    model.enableExternalScorer(vAR_LM_Path)

    model.setScorerAlphaBeta(vAR_lm_alpha,vAR_lm_beta)
    model.setBeamWidth(vAR_beam_width)
    
# Conversion of audio data to 16khz
    !ffmpeg -i hospital.wav -vn -ar 16000 -ac 1 hospital1.wav

    def read_wav_file(filename):
        with wave.open(filename, 'rb') as vAR_temp:       # "rb" mode opens the file in binary format for reading
            rate=vAR_temp.getframerate()
            frames=vAR_temp.getnframes()
            buffer=vAR_temp.readframes(frames)
        return buffer, rate

    def transcribe(file):
        buffer,rate=read_wav_file(file)
        data16=np.frombuffer(buffer,dtype=np.int16)
        return model.stt(data16)
    
# Storing the output in a .txt file    
    f= open(vAR_Text_Path,"w+")

    f.write(transcribe('hospital1.wav'))
    f.close()

# Step 4: Extracting Symptoms from text file

In [ ]:
def Clone_CliNER():
    git.Git("DeepsphereProjects/Healthcare/MedicalTranscription/4_ModelImplementation").clone("https://github.com/text-machine-lab/CliNER.git")

In [ ]:
def Predict_symptoms():
    %cd CliNER/
    !python cliner predict --txt DeepsphereProjects/Healthcare/MedicalTranscription/4_ModelImplementation/CliNER/data/examples/hospital111.txt --out data/predictions --model models/silver.crf --format i2b2

# Step 5: Text classification and visualization

In [ ]:
def Text_Classification_and_Visualization():
    with open(vAR_Text_Path,'r') as file:
        read_file = file.read()
# Text analytics (summarized content)
    tokenized_text=sent_tokenize(read_file)
    tokenized_word=word_tokenize(read_file)

    fdist = FreqDist(tokenized_word)
# removing all the stop words
    vAR_stop_words=set(stopwords.words("english"))
    filtered_sent=[]
    for vAR_iter in tokenized_word:
        if vAR_iter not in vAR_stop_words:
            filtered_sent.append(vAR_iter)
    with open(vAR_Symptom_Path, "r") as vAR:
        lines = vAR.readlines()
    Outputlist = []
# matched symptoms from filtered tokenized words from text file
    for vAR_val in filtered_sent:
        with open(vAR_Symptom_Path,"r") as vAR:
            lines = vAR.readlines()
        for line in lines:
            vAR_out = line.strip()
            if vAR_val == vAR_out:
                Outputlist.append(vAR_val)
# Plotting a Line Chart                
    plt.plot(Outputlist)
    plt.ylabel('Symptoms')
    plt.xlabel('numbers')
    plt.show()
    symptoms = list(set(Outputlist))
    print(symptoms)
    vAR_df = DataFrame (Outputlist,columns=['Symptoms'])
    vAR_table=vAR_df['Symptoms'].value_counts()
    
# visualization using Pie chart
    vAR_table.plot.pie(y=vAR_df.index,shadow=False, explode=None, startangle=90, autopct='%1.1f%%')
    plt.axis('equal')
    plt.tight_layout()
    plt.show()
    cf.go_offline()
    cf.set_config_file(offline=False, world_readable=True)
# visualization using cufflinks and plotly libraries
    vAR_df.iplot(kind='hist',bins=50,xTitle='symptoms',linecolor='black',yTitle='frequency',title='Symptoms frequency distribution')
    
if __name__ == "__main__":
    main()